<img align="right" width="200" height="200" src="https://static.tildacdn.com/tild6236-6337-4339-b337-313363643735/new_logo.png">

# Spark Structured Streaming I
**Андрей Титов**  
tenke.iu8@gmail.com  

## На этом занятии
+ Общие сведения
+ Rate streaming
+ File streaming
+ Kafka streaming

## Общие сведения

Системы поточной обработки данных:
- работают с непрерывным потоком данных
- нужно хранить состояние стрима
- результат обработки быстро появляется в целевой системе
- должны проектироваться с учетом требований к высокой доступности
- важная скорость обработки данных и время зажержки (лаг)

### Примеры систем поточной обработки данных

#### Карточный процессинг
- нельзя терять платежи
- нельзя дублировать платежи
- простой сервиса недопустим
- максимальное время задержки ~ 1 сек
- небольшой поток событий
- OLTP

#### Обработка логов безопасности
- потеря единичных событий допустима
- дублирование единичных событий допустимо
- простой сервиса допустим
- максимальное время задержки ~ 1 час
- большой поток событий
- OLAP

### Виды стриминг систем

#### Real-time streaming
- низкие задержки на обработку
- низкая пропускная способность
- подходят для критичных систем
- пособытийная обработка
- OLTP
- exactly once consistency (нет потери данных и нет дубликатов)

#### Micro batch streaming
- высокие задержки
- высокая пропускная способность
- не подходят для критичных систем
- обработка батчами
- OLAP
- at least once consistency (во время сбоев могут возникать дубликаты)

### Выводы:
+ Существуют два типа систем поточной обработки данных - real-time и micro-batch
+ Spark Structured Streaming является micro-batch системой
+ При работе с большими данными обычно пропускная способность важнее, чем время задержки


## Rate streaming

Самый простой способ создать стрим - использовать `rate` источник. Созданный DF является streaming, о чем нам говорит метод создания `readStream` и атрибут `isStreaming`. `rate` хорошо подходит для тестирования приложений, когда нет возможности подключится к потоку реальных данных

In [ ]:
spark

In [ ]:
sdf = spark.readStream.format("rate").load()
sdf

In [ ]:
sdf.isStreaming

У `sdf`, как и у любого DF, есть схема и план выполнения:

In [ ]:
sdf.printSchema()
sdf.explain(True)

В отличии от обычных DF, у `sdf` нет таких методов, как `show`, `collect`, `take`. Поэтому для того, чтобы посмотреть их содержимое, мы должны использовать `console` синк и создать `StreamingQuery`. Процессинг начинается только после вызова метода `start`. `trigger` позволяет настроить, как часто стрим будет читать новые данные и обрабатывать их

In [ ]:
def create_console_sink(df):
    return df \
            .writeStream \
            .format("console") \
            .trigger(processingTime="5 seconds") \
            .option("truncate", "false") \
            .option("numRows", "20")

In [ ]:
sink = create_console_sink(sdf)
sink

In [ ]:
sq = sink.start()
sq.isActive

Чтобы остановить DF, можно вызвать метод `stop` к `sdf`, либо получить список всех streming DF и остановить их:

In [ ]:
def kill_all():
    streams = SparkSession.builder.getOrCreate().streams.active
    for s in streams:
        desc = s.lastProgress["sources"][0]["description"]
        s.stop()
        print("Stopped {s}".format(s=desc))

In [ ]:
kill_all()

Создадим стрим, выполняющий запись в `parquet` файл:

In [ ]:
def create_parquet_sink(df, file_name):
    return df \
            .writeStream \
            .format("parquet") \
            .option("path", "datasets/{f}".format(f=file_name)) \
            .option("checkpointLocation", "chk/{f}".format(f=file_name)) \
            .trigger(processingTime="10 seconds")

In [ ]:
sink = create_parquet_sink(sdf, "s1.parquet")
sink

In [ ]:
sq = sink.start()
sq

In [ ]:
sq.isActive

In [ ]:
kill_all()

Убедимся, что стрим пишется в файл:

In [ ]:
!ls -alh datasets/s1.parquet

Прочитаем файл с помощью Spark:

In [ ]:
rates = spark.read.parquet("datasets/s1.parquet")
print(rates.count())
rates.printSchema()
rates.show(5, False)

Параллельно внутри одного Spark приложения может работать несколько стримов:

In [ ]:
console_sink = create_console_sink(sdf)
console_sq = console_sink.start()

In [ ]:
kill_all()

Напишем функцию, которая добавляет к нашей колонке случайный `ident` аэропорта из датасета [Airport Codes](https://datahub.io/core/airport-codes)  

In [ ]:
csv_options = {"header": "true", "inferSchema": "true"}
airports = spark.read.options(**csv_options).csv("datasets/airport-codes.csv")
airports.printSchema()
airports.show(1, 200, True)

In [ ]:
from pyspark.sql.functions import col
idents_rows = airports.select(col("ident")).limit(200).distinct().collect()
idents = [row["ident"] for row in idents_rows]
idents

In [ ]:
from pyspark.sql.functions import shuffle, array, lit
idents_cols = [lit(i) for i in idents]
idents_array = array(*idents_cols)
shuffled = shuffle(idents_array)
rand_ident = shuffled[0]

ident_sdf = sdf.withColumn("ident", rand_ident)
ident_sdf

In [ ]:
ident_pq_sink = create_parquet_sink(ident_sdf, "s2.parquet")
ident_pq_sq = ident_pq_sink.start()

Проверим, что данные записываются в `parquet`

In [ ]:
ident_pq = spark.read.parquet("datasets/s2.parquet")
print(ident_pq.count())
ident_pq.printSchema()
ident_pq.show(5, False)

Временно остановим стрим, он понадобится нам для следующих экспериментов

In [ ]:
kill_all()

### Выводы:
- `rate` - самый простой способ создать стрим для тестирования приложений
- стрим начинает работу после вызова метода `start` и не блокирует основной поток программы
- в одном Spark приложении может работать несколько стримов одновременно

## File Streaming
Spark позволяет запустить стрим, который будет "слушать" директорию и читать из нее новые файлы. При этом за раз будет прочитано количество файлов, установленное в параметре `maxFilesPerTrigger` [ссылка](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#input-sources). В этом кроется одна из основных проблем данного источника. Поскольку стрим, сконфигурированный под чтение небольших файлов, может "упасть", если в директорию начнут попадать файлы большого объема. Создадим стрим из директории `datasets/s2.parquet`:

In [ ]:
sdf_from_parquet = spark \
                    .readStream \
                    .format("parquet") \
                    .option("maxFilesPerTrigger", "1") \
                    .option("path", "datasets/s2.parquet") \
                    .load()

sdf_from_parquet.printSchema()

Поскольку в директорию могут попасть любые данные, а df должен иметь фиксированную схему, то Spark не позволяет нам создавать SDF на основе файлов без указания схемы.

In [ ]:
from pyspark.sql.functions import lower

sdf_from_parquet = spark \
                    .readStream \
                    .format("parquet") \
                    .schema(ident_pq.schema) \
                    .option("maxFilesPerTrigger", "10") \
                    .option("path", "datasets/s2.parquet") \
                    .load() \
                    .withColumn("ident", lower(col("ident")))

sdf_from_parquet.printSchema()

In [ ]:
console_sink = create_console_sink(sdf_from_parquet)
console_sink.start()

In [ ]:
kill_all()

File source позволяет со всеми типами файлов, с которыми умеет работать Spark: `parquet`, `orc`, `csv`, `json`, `text`.

### Выводы:
- Spark позволяет создавать SDF на базе всех поддерживаемых типов файлов
- При создании SDF вы должны указать схему данных
- File streaming имеет несколько серьезных недостатков:
  + Входной поток можно ограничить только макисмальным количество файлов, попадающих в батч
  + Если стрим упадает посередине файла, то при перезапуске эти данные будут обработаны еще раз

<img align="right" width="100" height="100" src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Apache_kafka.svg/1200px-Apache_kafka.svg.png">

## Kafka streaming

https://kafka.apache.org

**Apache Kafka** - самая распространенная в мире система, на основе которой строятся приложения для поточной обработки данных. Она имеет несколько преимуществ:
- высокая пропускная способность
- высокая доступность за счет распределенной архитектуры и репликации
- у каждого сообщения есть свой номер, который называется offset, что позволяет гранулярно сохранять состояние стрима

### Архитектура системы

#### Topic
Топик - это таблицы в Kafka. Мы пишем данные в топик и читаем данные из топика. Топик как правило распределен по нескольким узлам кластера для обеспечения высокой доступности и скорости работы с данными

<img align="center" width="500" height="500" src="https://kafka.apache.org/25/images/log_anatomy.png">

#### Partition
Партиции - это блоки, из которых состоят топики. Партиция представляет собой неделимый блок, который хранится на одном из узлов. Топик может иметь произвольное количество партиций. Чем больше партиций - тем выше параллелзим при чтении и записи, однако слишком большое число партиций в топике может привести к замедлению работы всей системы.

#### Replica
Каждая партиция имеет (может иметь) несколько реплик. Внешние приложения всегда работают (читают и пишут) с основной репликой. Остальные реплики являются дочерними и не используются во внешнем IO. Если узел, на котором расположена основная реплика, падает, то одна из дочерних реплик становится основной и работа с данными продолжается

#### Message
Сообщения - это данные, которые мы пишем и читаем в Kafka. Они представлены кортежем (Key, Value), но ключ может быть иметь значение `null` (используется не всегда). Сереализация и десереализация данных всегда происходит на уровне клиентов Kafka. Сама Kafka ничего о типах данных не знает и хранит ключи и значения в виде массива байт

#### Offset
Оффсет - это порядковый номер сообщения в партиции. Когда мы пишем сообщение (сообщение всегда пишется в одну из партиций топика), Kafka помещает его в топик с номер `n+1`, где `n` - номер последнего сообщения в этом топике

<img align="center" width="400" height="400" src="https://kafka.apache.org/25/images/log_consumer.png">

#### Producer
Producer - это приложение, которое пишет в топик. Producer'ов может быть много. Параллельная запись достигается за счет того, что каждое новое сообщение попадает в случайную партицию топика (если не указан `key`)

#### Consumer
Consumer - это приложение, читающее данные из топика. Consumer'ов может быть много, в этом случае они называются `consumer group`. Параллельное чтение достигается за счет распределения партиций топика между consumer'ами в рамках одной группы. Каждый consumer читает данные из "своих" партиций и ничего про другие не знает. Если consumer падает, то "его" партиции переходят другим consumer'ам.

#### Commit
Коммитом в Kafka называют сохранение информации о факте обработки сообщения с определенным оффсетом. Поскольку оффсеты для каждой партиции топика свои, то и информация о последнем обработанном оффсете хранится по каждой партиции отдельно. Обычные приложения пишут коммиты в специальный топик Kafka, который имеет название `__consumer_offsets`. Spark хранит обработанные оффсеты по каждому батчу в ФС (например, в HDFS).

#### Retention
Поскольку кластер Kafka не может хранить данные вечно, то в ее конфигурации задаются пороговые значение по **объему** и **времени хранения** для каждого топика, при превышении которых данные удаляются. Например, если у топика A установлен renention по времени 1 месяц, то данные будут хранится в системе не менее одного месяца (и затем будут удалены одной из внутренних подсистем)

### Spark connector
https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10  
https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html  

### Запуск Kafka в docker
```shell
docker run --rm \
   -p 2181:2181 \
   --name=test_zoo \
   -e ZOOKEEPER_CLIENT_PORT=2181 \
   confluentinc/cp-zookeeper
```

```shell
docker run --rm \
    -p 9092:9092 \
    --name=test_kafka \
    -e KAFKA_ZOOKEEPER_CONNECT=host.docker.internal:2181 \
    -e KAFKA_ADVERTISED_LISTENERS=PLAINTEXT://host.docker.internal:9092 \
    -e KAFKA_OFFSETS_TOPIC_REPLICATION_FACTOR=1 \
    confluentinc/cp-kafka
```

### Работа с Kafka с помощь Static Dataframe

Spark позволяет работать с кафкой как с обычной базой данных. Запишем данные в топик `test_topic0`. Для этого нам необходимо подготовить DF, в котором будет две колонки:
- `value: String` - данные, которые мы хотим записать
- `topic: String` - топик, куда писать каждую строку DF

In [ ]:
ident_pq = spark.read.parquet("datasets/s2.parquet")
print(ident_pq.count())
ident_pq.printSchema()
ident_pq.show(5, False)

In [ ]:
from pyspark.sql.functions import struct, to_json

def write_kafka(topic, data):
    kafka_params = {"kafka.bootstrap.servers": "localhost:9092"}
    kafka_doc = to_json(struct(*[col(c) for c in data.columns]))
    data \
        .select(kafka_doc.alias("value")) \
        .withColumn("topic", lit(topic)) \
        .write.format("kafka") \
        .options(**kafka_params).save()

In [ ]:
write_kafka("test_topic0", ident_pq)

Прочитаем данные из Kafka:

In [ ]:
kafka_params = {
    "kafka.bootstrap.servers": "localhost:9092",
    "subscribe": "test_topic0"
}

df = spark.read.format("kafka").options(**kafka_params).load()

df.printSchema()
df.show()

Чтение из Kafka имеет несколько особенностей:
- по умолчанию читается все содержимое топика. Поскольку обычно в нем много данных, эта операция может создать большую нагрузку на кластер Kafka и Spark приложение
- колонки `value` и `key` имеют тип `binary`, который необходимо десереализовать

In [ ]:
df.select(col("value").cast("string")).show(10, False)

Чтобы прочитать только определенную часть топика, нам необходимо задать минимальный и максимальный оффсет для чтения с помощью параметров `startingOffsets` , `endingOffsets`. Возьмем два случайных события:

In [ ]:
df.sample(0.1).limit(2).select(col("topic"), col("partition"), col("offset")).show()

На основании этих событий подготовим параметры `startingOffsets` и `endingOffsets`

In [ ]:
kafka_params = {
    "kafka.bootstrap.servers": "localhost:9092",
    "subscribe": "test_topic0",
    "startingOffsets": """ { "test_topic0": { "0": 6 } } """,
    "endingOffsets": """ { "test_topic0": { "0": 9 } }  """
}


df = spark.read.format("kafka").options(**kafka_params).load()

df.printSchema()
df.show(20)

По умолчанию параметр `startingOffsets` имеет значение `earliest`, а `endingOffsets` - `latest`. Поэтому, когда мы не указывали эти параметры, Spark прочитал содержимое всего топика

Чтобы получить наши данные, которые мы записали в топик, нам необходимо их десереализовать. В нашем случае достаточно использовать `.cast("string")`, однако это работает не всегда, т.к. формат данных может быть произвольным.

In [ ]:
json_doc = df.select(col("value").cast("string"))

json_doc.show(20, False)

### Работа с Kafka с помощью Streaming DF
При создании SDF из Kafka необходимо помнить, что:
- `startingOffsets` по умолчанию имеет значение `latest`
- `endingOffsets` использовать нельзя
- количество сообщений за батч можно (и нужно) ограничить параметром `maxOffsetPerTrigger` (по умолчанию он не задан и первый батч будет содержать данные всего топика

In [ ]:
killAll()

In [ ]:
kafka_params = {
    "kafka.bootstrap.servers": "localhost:9092",
    "subscribe": "test_topic0",
    "startingOffsets": """earliest""",
    "maxOffsetsPerTrigger": "5"
}

sdf = spark.readStream.format("kafka").options(**kafka_params).load()
parsed_sdf = sdf.select(col("value").cast("string"), col("topic"), col("partition"), col("offset"))

sink = create_console_sink(parsed_sdf)

sq = sink.start()

Если мы перезапустим этот стрим, он повторно прочитает все данные. Чтобы обеспечить сохранение состояния стрима после обработки каждого батча, нам необходимо добавить параметр `checkpointLocation` в опции `writeStream`:

In [ ]:
def create_console_sink_with_checkpoint(chk_name, df): 
    return df \
        .writeStream \
        .format("console") \
        .trigger(processingTime="10 seconds") \
        .option("checkpointLocation", "chk/{n}".format(n=chk_name)) \
        .option("truncate", "false") \
        .option("numRows", "20")

In [ ]:
sink = create_console_sink_with_checkpoint("test0", parsed_sdf)
sq = sink.start()

In [ ]:
kill_all()

### Выводы:
- Apache Kafka - распределенная система, обеспечивающая передачу потока данных в слабосвязанных системах
- Работать с Kafka можно как с использованием Static DF, так и с помощью Streaming DF
- Чтобы стрим запоминал свое состояние после остановки, необходимо использовать checkpoint - директорию на HDFS (или локальной ФС), в которую будет сохранятся состояние стрима после каждого батча

В конце работы не забудьте остановить Spark:

In [ ]:
spark.stop()